## universo  (34,380 tickers - activos + inactivos)

**Objetivo**: descargar universo completo 2004-2025 (34,380 tickers - activos + inactivos). Ese archivo es el primer paso del pipeline de Small Caps. Sirve para descargar el universo completo de compañías (activas + delistadas) de Polygon.io y dejarlo guardado como snapshot diario en formato Parquet particionado.  
**Script:** [scripts/ingest_reference_universe.py](../scripts/00_universe_ingest/download_universe.py)      
**Fuente de datos**: Polygon API (tickers snapshot histórico)   

In [2]:
import polars as pl
import os

pl.Config.set_tbl_rows(-1)  # muestra todas las filas
pl.Config.set_tbl_cols(-1)  # muestra todas las columnas
pl.Config.set_tbl_width_chars(200)  # ancho máximo (ajusta si se corta)
pl.Config.set_tbl_hide_column_data_types(True)  # opcional: oculta tipos

polars.config.Config

In [3]:


# Cambiar al directorio correcto del proyecto actual
os.chdir(r"D:\TSIS_SmallCaps")

# 1. tickers_all.parquet
print("\n📊 1. tickers_all.parquet")
print("-"*100)
df_all = pl.read_parquet("raw/polygon/reference/tickers_snapshot/snapshot_date=2025-11-01/tickers_all.parquet")
print(f"Total rows: {len(df_all):,}")




# 2. tickers_active.parquet
print("\n📊 2. tickers_active.parquet (11,853 active tickers)")
print("-"*100)
df_active = pl.read_parquet("raw/polygon/reference/tickers_snapshot/snapshot_date=2025-11-01/tickers_active.parquet")
print(f"Total rows: {len(df_active):,}")
print(f"\nHEAD(2):")
print(df_all.head(2).transpose(include_header=True))



# 3. tickers_inactive.parquet
print("\n📊 3. tickers_inactive.parquet (22,557 inactive tickers)")
print("-"*100)
df_inactive = pl.read_parquet("raw/polygon/reference/tickers_snapshot/snapshot_date=2025-11-01/tickers_inactive.parquet")
print(f"Total rows: {len(df_inactive):,}")
print(f"\nHEAD(2):")
print(df_all.head(2).transpose(include_header=True))


📊 1. tickers_all.parquet
----------------------------------------------------------------------------------------------------
Total rows: 34,324

📊 2. tickers_active.parquet (11,853 active tickers)
----------------------------------------------------------------------------------------------------
Total rows: 11,899

HEAD(2):
shape: (14, 3)
┌──────────────────┬────────────────────────────────┬────────────────────────────────┐
│ column           ┆ column_0                       ┆ column_1                       │
╞══════════════════╪════════════════════════════════╪════════════════════════════════╡
│ ticker           ┆ A                              ┆ AA                             │
│ name             ┆ Agilent Technologies Inc.      ┆ Alcoa Corporation              │
│ market           ┆ stocks                         ┆ stocks                         │
│ locale           ┆ us                             ┆ us                             │
│ primary_exchange ┆ XNYS                      

## filtra los tickers que estuvieron listados entre 2019 y 2025

No descarga nada: toma el snapshot ya generado (por el script anterior) y filtra los tickers que estuvieron listados entre 2019 y 2025.

In [4]:
# 4. tickers_2019_2025.parquet - Universo Filtrado 2019-2025
print("\n📊 4. tickers_2019_2025.parquet (20,471 tickers filtrados)")
print("-"*100)
df_2019_2025 = pl.read_parquet("processed/universe/tickers_2019_2025.parquet")
print(f"Total rows: {len(df_2019_2025):,}")
print(f"Total columns: {len(df_2019_2025.columns)}")

# Estadísticas por active
print(f"\n📈 Distribución por Status:")
print(df_2019_2025.group_by("active").len().sort("active"))

# Estadísticas por exchange
print(f"\n📈 Distribución por Exchange:")
print(df_2019_2025.group_by("primary_exchange").len().sort("len", descending=True).head(10))

# Estadísticas por type
print(f"\n📈 Distribución por Type:")
print(df_2019_2025.group_by("type").len().sort("len", descending=True))

# Verificar delisted_utc
n_delisted = len(df_2019_2025.filter(pl.col("delisted_utc").is_not_null()))
print(f"\n📈 Con delisted_utc: {n_delisted:,} ({n_delisted/len(df_2019_2025)*100:.1f}%)")

# ====== EJEMPLOS DE COMPAÑÍAS ======

# Ejemplo de compañía ACTIVA (head(1) transpuesto)
print(f"\n📋 Ejemplo de Compañía ACTIVA:")
print("-"*100)
df_active_example = df_2019_2025.filter(pl.col("active") == True).head(1)
print(df_active_example.transpose(include_header=True))

# Ejemplo de compañía INACTIVA (head(1) transpuesto)
print(f"\n📋 Ejemplo de Compañía INACTIVA:")
print("-"*100)
df_inactive_example = df_2019_2025.filter(pl.col("active") == False).head(1)
print(df_inactive_example.transpose(include_header=True))

# ====== PREPARAR DATOS CON FECHAS ======

from datetime import date

# Convertir delisted_utc a fecha
if "delisted_utc" in df_2019_2025.columns:
    df_with_dates = df_2019_2025.with_columns(
        pl.col("delisted_utc").str.slice(0, 10).str.strptime(pl.Date, "%Y-%m-%d", strict=False).alias("delisted_date")
    )
else:
    df_with_dates = df_2019_2025.with_columns(pl.lit(None).alias("delisted_date"))

# ====== DISTRIBUCIÓN POR AÑO - ACTIVOS ======

print(f"\n📊 Distribución por año (Activos - cuántos estaban activos cada año):")
print("-"*100)

active_years = []
active_counts = []

for year in range(2019, 2026):
    year_end = date(year, 12, 31)
    
    # Activos en ese año = delisted_date is null O delisted_date > year_end
    count = len(df_with_dates.filter(
        pl.col("delisted_date").is_null() | 
        (pl.col("delisted_date") > year_end)
    ))
    
    active_years.append(year)
    active_counts.append(count)

df_active_dist = pl.DataFrame({
    "active_year": active_years,
    "len": active_counts
})

print(f"Total activos hoy: {len(df_2019_2025.filter(pl.col('active') == True)):,}")
print(df_active_dist)

# ====== DISTRIBUCIÓN POR AÑO - DESLISTADOS ======

print(f"\n📊 Distribución por año (Deslistados - cuántos fueron deslistados cada año):")
print("-"*100)

delist_years = []
delist_counts = []

for year in range(2019, 2026):
    year_start = date(year, 1, 1)
    year_end = date(year, 12, 31)
    
    # Deslistados en ese año = delisted_date between year_start and year_end
    count = len(df_with_dates.filter(
        (pl.col("delisted_date").is_not_null()) &
        (pl.col("delisted_date") >= year_start) &
        (pl.col("delisted_date") <= year_end)
    ))
    
    delist_years.append(year)
    delist_counts.append(count)

df_delist_dist = pl.DataFrame({
    "delist_year": delist_years,
    "len": delist_counts
})

print(f"Total inactivos con fecha de delist: {len(df_with_dates.filter(pl.col('delisted_date').is_not_null())):,}")
print(df_delist_dist)

print(f"\nColumnas disponibles:")
print(df_2019_2025.columns)


📊 4. tickers_2019_2025.parquet (20,471 tickers filtrados)
----------------------------------------------------------------------------------------------------
Total rows: 20,471
Total columns: 15

📈 Distribución por Status:
shape: (2, 2)
┌────────┬───────┐
│ active ┆ len   │
╞════════╪═══════╡
│ false  ┆ 8574  │
│ true   ┆ 11897 │
└────────┴───────┘

📈 Distribución por Exchange:
shape: (5, 2)
┌──────────────────┬──────┐
│ primary_exchange ┆ len  │
╞══════════════════╪══════╡
│ XNAS             ┆ 9796 │
│ XNYS             ┆ 5451 │
│ ARCX             ┆ 3258 │
│ BATS             ┆ 1348 │
│ XASE             ┆ 618  │
└──────────────────┴──────┘

📈 Distribución por Type:
shape: (15, 2)
┌─────────┬──────┐
│ type    ┆ len  │
╞═════════╪══════╡
│ CS      ┆ 8698 │
│ ETF     ┆ 5475 │
│ WARRANT ┆ 1869 │
│ UNIT    ┆ 1239 │
│ PFD     ┆ 953  │
│ ADRC    ┆ 616  │
│ FUND    ┆ 519  │
│ RIGHT   ┆ 306  │
│ SP      ┆ 306  │
│ ETN     ┆ 178  │
│ ETS     ┆ 139  │
│ null    ┆ 91   │
│ ETV     ┆ 75   │
│ INDE

## Filtrado por CS (Stocks) + XNAS/XNYS (Exchange)

* **Objetivo**: Filtrar únicamente Common Stocks (type=CS) listados en NASDAQ (XNAS) o NYSE (XNYS) del universo temporal 2019-2025. Este filtro excluye ETFs, Warrants, Units, Preferred Stocks y otros tipos de instrumentos, así como tickers en exchanges menores (ARCX, BATS, XASE).  
* **Fuente de datos**: Universo filtrado temporalmente 2019-2025 (20,471 tickers)  
* **Script**: [scripts/filter_universe_cs_exchanges.py](../../../scripts/00_universe_ingest/filter_universe_cs_exchanges.py)  
* **Input**: `processed/universe/tickers_2019_2025.parquet` (20,471 tickers)  
* **Filtros aplicados**:  
    - `type = "CS"` (Common Stock)  
    - `primary_exchange IN ["XNAS", "XNYS"]` (NASDAQ o NYSE)  

In [5]:
# ==============================================================================
# 5. tickers_2019_2025_cs_exchanges.parquet - Filtrado por CS + XNAS/XNYS
# ==============================================================================

print("\n" + "="*100)
print("📊 5. tickers_2019_2025_cs_exchanges.parquet - Common Stocks en NASDAQ/NYSE")
print("="*100)

df_cs_exchanges = pl.read_parquet("processed/universe/tickers_2019_2025_cs_exchanges.parquet")

# Estadísticas básicas
print(f"\n✅ Total tickers CS en XNAS/XNYS: {len(df_cs_exchanges):,}")
print(f"   Columnas disponibles: {len(df_cs_exchanges.columns)}")
print(f"\n📋 Columnas:")
for i, col in enumerate(df_cs_exchanges.columns, 1):
    print(f"   {i:2d}. {col}")

# Distribución activos/inactivos
n_active = len(df_cs_exchanges.filter(pl.col("active") == True))
n_inactive = len(df_cs_exchanges.filter(pl.col("active") == False))
print(f"\n📈 Distribución:")
print(f"   Activos hoy:   {n_active:,} ({n_active/len(df_cs_exchanges)*100:.1f}%)")
print(f"   Inactivos hoy: {n_inactive:,} ({n_inactive/len(df_cs_exchanges)*100:.1f}%)")

# Distribución por exchange
print(f"\n🏛️  Distribución por Exchange:")
exchange_dist = df_cs_exchanges.group_by("primary_exchange").len().sort("len", descending=True)
for row in exchange_dist.iter_rows(named=True):
    pct = row['len'] / len(df_cs_exchanges) * 100
    print(f"   {row['primary_exchange']:6s}: {row['len']:,} ({pct:.1f}%)")

# Verificar que SOLO hay CS
print(f"\n✅ Verificación de tipo:")
type_dist = df_cs_exchanges.group_by("type").len().sort("len", descending=True)
for row in type_dist.iter_rows(named=True):
    print(f"   {row['type']}: {row['len']:,}")

# EJEMPLOS DE COMPAÑÍAS
print(f"\n" + "-"*100)
print("📋 EJEMPLO: Compañía ACTIVA (CS en NASDAQ/NYSE)")
print("-"*100)
df_active_example = df_cs_exchanges.filter(pl.col("active") == True).head(1)
print(df_active_example.transpose(include_header=True))

print(f"\n" + "-"*100)
print("📋 EJEMPLO: Compañía INACTIVA (CS en NASDAQ/NYSE)")
print("-"*100)
df_inactive_example = df_cs_exchanges.filter(pl.col("active") == False).head(1)
print(df_inactive_example.transpose(include_header=True))

# DISTRIBUCIÓN POR AÑO - ACTIVOS
print(f"\n" + "-"*100)
print("📊 Distribución por año - ACTIVOS (tickers que estaban activos en cada año)")
print("-"*100)

from datetime import date

# Convertir delisted_utc a Date
df_with_dates = df_cs_exchanges.with_columns(
    pl.col("delisted_utc").str.slice(0, 10).str.strptime(pl.Date, "%Y-%m-%d", strict=False).alias("delisted_date")
)

# Para activos: contar los que NO estaban delistados en ese año
active_years = []
active_counts = []

for year in range(2019, 2026):
    year_end = date(year, 12, 31)
    # Activos en year = delisted_date is NULL OR delisted_date > year_end
    count = len(df_with_dates.filter(
        pl.col("delisted_date").is_null() | 
        (pl.col("delisted_date") > year_end)
    ))
    active_years.append(year)
    active_counts.append(count)

df_active_by_year = pl.DataFrame({
    "year": active_years,
    "activos": active_counts
})

print(df_active_by_year)

# DISTRIBUCIÓN POR AÑO - DESLISTADOS
print(f"\n" + "-"*100)
print("📊 Distribución por año - DESLISTADOS (tickers delistados en cada año)")
print("-"*100)

delisted_years = []
delisted_counts = []

for year in range(2019, 2026):
    year_start = date(year, 1, 1)
    year_end = date(year, 12, 31)
    # Deslistados en year = delisted_date BETWEEN year_start AND year_end
    count = len(df_with_dates.filter(
        (pl.col("delisted_date").is_not_null()) &
        (pl.col("delisted_date") >= year_start) &
        (pl.col("delisted_date") <= year_end)
    ))
    delisted_years.append(year)
    delisted_counts.append(count)

df_delisted_by_year = pl.DataFrame({
    "year": delisted_years,
    "deslistados": delisted_counts
})

print(df_delisted_by_year)

# RESUMEN COMPARATIVO
print(f"\n" + "="*100)
print("📊 RESUMEN COMPARATIVO - Pipeline de Filtrado")
print("="*100)

pipeline_summary = pl.DataFrame({
    "Paso": [
        "1. Snapshot completo (2025-11-01)",
        "2. Filtro temporal (2019-2025)",
        "3. Filtro CS + XNAS/XNYS"
    ],
    "Tickers": [34_324, 20_471, 8_307],
    "Activos": [11_899, 11_897, 5_007],
    "Inactivos": [22_557, 8_574, 3_300]
}).with_columns([
    (pl.col("Tickers") / 34324 * 100).round(1).alias("% del total"),
])

print(pipeline_summary)

print(f"\n✅ Filtrado completado exitosamente!")
print(f"   De {34_324:,} tickers iniciales → {8_307:,} Common Stocks en NASDAQ/NYSE (2019-2025)")
print(f"   Retención: {8_307/34_324*100:.1f}%")


📊 5. tickers_2019_2025_cs_exchanges.parquet - Common Stocks en NASDAQ/NYSE

✅ Total tickers CS en XNAS/XNYS: 8,307
   Columnas disponibles: 15

📋 Columnas:
    1. ticker
    2. name
    3. market
    4. locale
    5. primary_exchange
    6. type
    7. active
    8. currency_name
    9. cik
   10. composite_figi
   11. share_class_figi
   12. last_updated_utc
   13. snapshot_date
   14. delisted_utc
   15. delisted_date

📈 Distribución:
   Activos hoy:   5,007 (60.3%)
   Inactivos hoy: 3,300 (39.7%)

🏛️  Distribución por Exchange:
   XNAS  : 5,619 (67.6%)
   XNYS  : 2,688 (32.4%)

✅ Verificación de tipo:
   CS: 8,307

----------------------------------------------------------------------------------------------------
📋 EJEMPLO: Compañía ACTIVA (CS en NASDAQ/NYSE)
----------------------------------------------------------------------------------------------------
shape: (15, 2)
┌──────────────────┬────────────────────────────────┐
│ column           ┆ column_0                       │
╞

## Universo Híbrido Enriquecido (Market Cap, Description, Employees, SIC Code, Delisted UTC)

* **Objetivo**: Enriquecer el universo filtrado CS+NASDAQ/NYSE con datos corporativos mediante una estrategia dual de enriquecimiento.  
Esta estrategia permite filtrar small caps activos (< $2B) sin perder tickers inactivos críticos para eliminar survivorship bias. 

* **Estrategia Dual de Enriquecimiento**:
    * Enriquecer para **ACTIVOS** (5,007 tickers): Obtienen datos desde `/v3/reference/tickers/{ticker}` (ticker_details API)
        - `market_cap` - Capitalización de mercado (para filtrar small caps < $2B)
        - `description` - Descripción del negocio
        - `weighted_shares_outstanding` - Acciones en circulación
        - `total_employees` - Número de empleados
        - `sic_code` / `sic_description` - Código de industria
        - `homepage_url` - Sitio web corporativo
        - `phone_number`, `address` - Contacto

    * Enriquecer para **INACTIVOS** (3,300 tickers):  NO tienen datos en ticker_details (404), pero SÍ tienen `delisted_utc` en snapshot original  
        - `delisted_utc` - Fecha de delisting (CRÍTICO para ML feature engineering)
        - `cik` - SEC CIK number
        - `composite_figi` - FIGI identifier
        - `list_date` - Fecha de listado original

In [7]:
# ============================================================================
# Verificación: Universo Híbrido Enriquecido
# ============================================================================
import polars as pl
from pathlib import Path

print("="*80)
print("VERIFICACIÓN: UNIVERSO HÍBRIDO ENRIQUECIDO")
print("="*80)

# Cargar archivo enriquecido
enriched_path = Path("processed/universe/hybrid_enriched_2025-11-01.parquet")
df = pl.read_parquet(enriched_path)

print(f"\nArchivo: {enriched_path}")
print(f"   Tamaño: {enriched_path.stat().st_size / 1024 / 1024:.2f} MB")

# Estadísticas generales
print(f"\n📊 ESTADÍSTICAS GENERALES")
print(f"   Total tickers: {len(df):,}")
print(f"   Total columnas: {len(df.columns)}")

# Separar activos e inactivos
df_active = df.filter(pl.col("active") == True)
df_inactive = df.filter(pl.col("active") == False)

print(f"\n   Activos: {len(df_active):,} ({len(df_active)/len(df)*100:.1f}%)")
print(f"   Inactivos: {len(df_inactive):,} ({len(df_inactive)/len(df)*100:.1f}%)")

# ============================================================================
# VERIFICAR ESTRATEGIA DUAL DE ENRIQUECIMIENTO
# ============================================================================

print(f"\n{'='*80}")
print("ESTRATEGIA DUAL DE ENRIQUECIMIENTO")
print(f"{'='*80}")

# ACTIVOS: Enriquecimiento desde ticker_details
print(f"\nACTIVOS ({len(df_active):,} tickers) - Desde ticker_details API:")
print(f"{'   Campo':<35} {'No-Null':<15} {'% Completitud'}")
print(f"   {'-'*70}")

active_fields = {
    "market_cap": "Market Cap",
    "weighted_shares_outstanding": "Shares Outstanding",
    "description": "Description",
    "total_employees": "Employees",
    "sic_code": "SIC Code",
    "homepage_url": "Homepage URL",
    "phone_number": "Phone Number",
}

for field, label in active_fields.items():
    if field in df_active.columns:
        non_null = len(df_active.filter(~pl.col(field).is_null()))
        pct = non_null / len(df_active) * 100
        print(f"   {label:<35} {non_null:>6,}/{len(df_active):<6,} {pct:>6.1f}%")

# INACTIVOS: Enriquecimiento desde snapshot original
print(f"\nINACTIVOS ({len(df_inactive):,} tickers) - Desde snapshot original:")
print(f"{'   Campo':<35} {'No-Null':<15} {'% Completitud'}")
print(f"   {'-'*70}")

inactive_fields = {
    "delisted_utc": "Delisted UTC (CRÍTICO ML)",
    "cik": "CIK",
    "composite_figi": "FIGI",
    "list_date": "List Date",
}

for field, label in inactive_fields.items():
    if field in df_inactive.columns:
        non_null = len(df_inactive.filter(~pl.col(field).is_null()))
        pct = non_null / len(df_inactive) * 100
        print(f"   {label:<35} {non_null:>6,}/{len(df_inactive):<6,} {pct:>6.1f}%")

# ============================================================================
# TABLA DE COMPLETITUD CONSOLIDADA
# ============================================================================

print(f"\n{'='*80}")
print("TABLA DE COMPLETITUD CONSOLIDADA")
print(f"{'='*80}\n")

# Crear tabla consolidada
completeness_data = []

# Market cap (solo activos)
if "market_cap" in df.columns:
    non_null_active = len(df_active.filter(~pl.col("market_cap").is_null()))
    pct = non_null_active / len(df_active) * 100
    completeness_data.append({
        "Campo": "market_cap",
        "No-Null": f"{non_null_active:,} / {len(df_active):,}",
        "% Completitud": f"{pct:.1f}%",
        "Tipo Ticker": "ACTIVOS",
        "Fuente": "ticker_details"
    })

# Delisted UTC (solo inactivos)
if "delisted_utc" in df.columns:
    non_null_inactive = len(df_inactive.filter(~pl.col("delisted_utc").is_null()))
    pct = non_null_inactive / len(df_inactive) * 100
    completeness_data.append({
        "Campo": "delisted_utc",
        "No-Null": f"{non_null_inactive:,} / {len(df_inactive):,}",
        "% Completitud": f"{pct:.1f}%",
        "Tipo Ticker": "INACTIVOS",
        "Fuente": "snapshot original"
    })

# Otros campos (activos)
other_active_fields = [
    "weighted_shares_outstanding",
    "description",
    "sic_code",
    "homepage_url",
    "total_employees",
]

for field in other_active_fields:
    if field in df.columns:
        non_null = len(df_active.filter(~pl.col(field).is_null()))
        pct = non_null / len(df_active) * 100
        completeness_data.append({
            "Campo": field,
            "No-Null": f"{non_null:,} / {len(df_active):,}",
            "% Completitud": f"{pct:.1f}%",
            "Tipo Ticker": "ACTIVOS",
            "Fuente": "ticker_details"
        })

# CIK y FIGI (todos)
for field in ["cik", "composite_figi"]:
    if field in df.columns:
        non_null = len(df.filter(~pl.col(field).is_null()))
        pct = non_null / len(df) * 100
        completeness_data.append({
            "Campo": field,
            "No-Null": f"{non_null:,} / {len(df):,}",
            "% Completitud": f"{pct:.1f}%",
            "Tipo Ticker": "TODOS",
            "Fuente": "snapshot original"
        })

# Imprimir tabla
df_completeness = pl.DataFrame(completeness_data)
print(df_completeness)

# ============================================================================
# VERIFICACIÓN CRÍTICA: SMALL CAPS
# ============================================================================

print(f"\n{'='*80}")
print("VERIFICACIÓN: SMALL CAPS (< $2B)")
print(f"{'='*80}\n")

if "market_cap" in df.columns:
    df_active_with_mcap = df_active.filter(~pl.col("market_cap").is_null())
    
    print(f"Activos con market_cap: {len(df_active_with_mcap):,} / {len(df_active):,}")
    
    # Small caps
    df_small_caps = df_active_with_mcap.filter(pl.col("market_cap") < 2_000_000_000)
    pct_small = len(df_small_caps) / len(df_active_with_mcap) * 100
    
    print(f"\nSmall caps (< $2B):")
    print(f"   Total: {len(df_small_caps):,} / {len(df_active_with_mcap):,} ({pct_small:.1f}%)")
    
    # Distribución por rangos
    print(f"\n   Distribución por rangos:")
    ranges = [
        ("< $300M", 0, 300_000_000),
        ("$300M - $1B", 300_000_000, 1_000_000_000),
        ("$1B - $2B", 1_000_000_000, 2_000_000_000),
        ("> $2B", 2_000_000_000, float('inf')),
    ]
    
    for label, min_cap, max_cap in ranges:
        count = len(df_active_with_mcap.filter(
            (pl.col("market_cap") >= min_cap) & (pl.col("market_cap") < max_cap)
        ))
        pct = count / len(df_active_with_mcap) * 100
        print(f"   {label:<15} {count:>5,} ({pct:>5.1f}%)")

# ============================================================================
# VERIFICACIÓN: SURVIVORSHIP BIAS ELIMINADO
# ============================================================================

print(f"\n{'='*80}")
print("VERIFICACIÓN: SURVIVORSHIP BIAS ELIMINADO")
print(f"{'='*80}\n")

if "delisted_utc" in df.columns:
    df_inactive_with_delisted = df_inactive.filter(~pl.col("delisted_utc").is_null())
    
    print(f"✅ Inactivos preservados: {len(df_inactive):,}")
    print(f"✅ Con delisted_utc: {len(df_inactive_with_delisted):,} / {len(df_inactive):,} ({len(df_inactive_with_delisted)/len(df_inactive)*100:.1f}%)")
    print(f"\n   Esto permite:")
    print(f"   • Calcular days_to_delisting (feature ML)")
    print(f"   • Detectar pump & dumps terminales")
    print(f"   • Eliminar survivorship bias en backtesting")

# ============================================================================
# RESUMEN FINAL
# ============================================================================

print(f"\n{'='*80}")
print("✅ RESUMEN FINAL")
print(f"{'='*80}\n")

print(f"Total tickers enriquecidos: {len(df):,}")
print(f"   • Activos con market_cap: {len(df_active.filter(~pl.col('market_cap').is_null())):,} ({len(df_active.filter(~pl.col('market_cap').is_null()))/len(df_active)*100:.1f}%)")
print(f"   • Inactivos con delisted_utc: {len(df_inactive.filter(~pl.col('delisted_utc').is_null())):,} ({len(df_inactive.filter(~pl.col('delisted_utc').is_null()))/len(df_inactive)*100:.1f}%)")
print(f"   • Small caps identificados: {len(df_small_caps):,}")
print(f"   • Sin survivorship bias: ✅")

print(f"\n{'='*80}")
print("Status: ✅ UNIVERSO ENRIQUECIDO - Listo para Fase B (Ingesta Daily/Minute)")
print(f"{'='*80}\n")

VERIFICACIÓN: UNIVERSO HÍBRIDO ENRIQUECIDO

Archivo: processed\universe\hybrid_enriched_2025-11-01.parquet
   Tamaño: 1.39 MB

📊 ESTADÍSTICAS GENERALES
   Total tickers: 8,307
   Total columnas: 25

   Activos: 5,007 (60.3%)
   Inactivos: 3,300 (39.7%)

ESTRATEGIA DUAL DE ENRIQUECIMIENTO

ACTIVOS (5,007 tickers) - Desde ticker_details API:
   Campo                            No-Null         % Completitud
   ----------------------------------------------------------------------
   Market Cap                           4,883/5,007    97.5%
   Shares Outstanding                   4,900/5,007    97.9%
   Description                          4,934/5,007    98.5%
   Employees                            4,509/5,007    90.1%
   SIC Code                             4,134/5,007    82.6%
   Homepage URL                         4,797/5,007    95.8%
   Phone Number                         4,204/5,007    84.0%

INACTIVOS (3,300 tickers) - Desde snapshot original:
   Campo                            N

## Filtro Small Caps Target Population (< $2B) con Anti-Survivorship Bias

* **Objetivo**: Filtrar la población target de Small Caps (market cap < $2B) preservando TODOS los tickers inactivos para eliminar survivorship bias. Este filtro dual aplica el umbral de capitalización SOLO a activos, mientras mantiene el 100% de inactivos como contrapartes históricas necesarias para ML.
* **Fuente de datos**: Universo enriquecido CS+XNAS/XNYS (8,307 tickers con market_cap y delisted_utc)
* **Script**: [scripts/filter_smallcaps_population.py](../../scripts/00_universe_ingest/filter_smallcaps_population.py)
* **Input**: `processed/universe/hybrid_enriched_2025-11-01.parquet` (8,307 tickers enriquecidos)

In [10]:
# ============================================================================
# Verificación: Universo Small Caps Target Population
# ============================================================================
import polars as pl

print("="*80)
print("VERIFICACIÓN: SMALL CAPS TARGET POPULATION (< $2B)")
print("="*80)

# Cargar universo Small Caps
df = pl.read_parquet("processed/universe/smallcaps_universe_2025-11-01.parquet")

print(f"\n📊 RESUMEN GENERAL")
print(f"   Total tickers: {len(df):,}")
print(f"   Total columnas: {len(df.columns)}")

# Separar activos e inactivos
df_active = df.filter(pl.col("active") == True)
df_inactive = df.filter(pl.col("active") == False)

print(f"\n   Small Caps activos: {len(df_active):,} ({len(df_active)/len(df)*100:.1f}%)")
print(f"   Inactivos preservados: {len(df_inactive):,} ({len(df_inactive)/len(df)*100:.1f}%)")

# ============================================================================
# MARKET CAP - SMALL CAPS ACTIVOS
# ============================================================================

print(f"\n{'='*80}")
print("MARKET CAP - SMALL CAPS ACTIVOS (< $2B)")
print(f"{'='*80}\n")

df_active_with_mcap = df_active.filter(~pl.col("market_cap").is_null())

print(f"Activos con market_cap: {len(df_active_with_mcap):,} / {len(df_active):,}")

# Estadísticas
mcap_stats = df_active_with_mcap.select("market_cap").describe()
print(f"\nEstadísticas:")
print(f"   Median: ${mcap_stats['market_cap'][5]:>15,.0f}")
print(f"   Mean:   ${mcap_stats['market_cap'][1]:>15,.0f}")
print(f"   Min:    ${mcap_stats['market_cap'][3]:>15,.0f}")
print(f"   Max:    ${mcap_stats['market_cap'][7]:>15,.0f}")

# Distribución por rangos
print(f"\nDistribución por rangos:")
ranges = [
    ("Micro caps (< $300M)", 0, 300_000_000),
    ("Small caps ($300M-$1B)", 300_000_000, 1_000_000_000),
    ("Mid-small caps ($1B-$2B)", 1_000_000_000, 2_000_000_000),
]

for label, min_cap, max_cap in ranges:
    count = len(df_active_with_mcap.filter(
        (pl.col("market_cap") >= min_cap) & (pl.col("market_cap") < max_cap)
    ))
    pct = count / len(df_active_with_mcap) * 100
    print(f"   {label:<30} {count:>5,} ({pct:>5.1f}%)")

# ============================================================================
# DELISTED UTC - INACTIVOS PRESERVADOS
# ============================================================================

print(f"\n{'='*80}")
print("DELISTED UTC - INACTIVOS PRESERVADOS (ANTI-SURVIVORSHIP BIAS)")
print(f"{'='*80}\n")

df_inactive_with_delisted = df_inactive.filter(~pl.col("delisted_utc").is_null())

print(f"Inactivos con delisted_utc: {len(df_inactive_with_delisted):,} / {len(df_inactive):,} ({len(df_inactive_with_delisted)/len(df_inactive)*100:.1f}%)")

# Años de delisting
if len(df_inactive_with_delisted) > 0:
    df_delisted_years = (
        df_inactive_with_delisted
        .with_columns(
            pl.col("delisted_utc").str.slice(0, 4).alias("delisted_year")
        )
        .group_by("delisted_year")
        .len()
        .sort("delisted_year")
    )
    
    print(f"\nDistribución por año de delisting:")
    for row in df_delisted_years.iter_rows(named=True):
        print(f"   {row['delisted_year']}: {row['len']:,}")

# ============================================================================
# DISTRIBUCIÓN POR EXCHANGE
# ============================================================================

print(f"\n{'='*80}")
print("DISTRIBUCIÓN POR EXCHANGE")
print(f"{'='*80}\n")

for exchange in ["XNAS", "XNYS"]:
    count = len(df.filter(pl.col("primary_exchange") == exchange))
    count_active = len(df_active.filter(pl.col("primary_exchange") == exchange))
    count_inactive = len(df_inactive.filter(pl.col("primary_exchange") == exchange))
    pct = count / len(df) * 100
    print(f"{exchange}:")
    print(f"   Total: {count:,} ({pct:.1f}%)")
    print(f"   Activos: {count_active:,} | Inactivos: {count_inactive:,}\n")

# ============================================================================
# COLUMNAS DISPONIBLES
# ============================================================================

print(f"{'='*80}")
print("COLUMNAS DISPONIBLES EN EL DATASET")
print(f"{'='*80}\n")

print(f"Total columnas: {len(df.columns)}")
print("\nColumnas principales:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")

# ============================================================================
# EJEMPLOS DE TICKERS
# ============================================================================

print(f"\n{'='*80}")
print("EJEMPLOS DE TICKERS")
print(f"{'='*80}\n")

# Micro cap activo
print("EJEMPLO 1: Micro Cap ACTIVO (< $300M)")
example_micro = df_active_with_mcap.filter(
    pl.col("market_cap") < 300_000_000
).head(1)
print(example_micro.select(["ticker", "name", "primary_exchange", "market_cap"]))

# Small cap activo
print("\nEJEMPLO 2: Small Cap ACTIVO ($300M-$1B)")
example_small = df_active_with_mcap.filter(
    (pl.col("market_cap") >= 300_000_000) & (pl.col("market_cap") < 1_000_000_000)
).head(1)
print(example_small.select(["ticker", "name", "primary_exchange", "market_cap"]))

# Inactivo reciente
print("\nEJEMPLO 3: INACTIVO RECIENTE (delisted 2024+)")
example_inactive = df_inactive_with_delisted.filter(
    pl.col("delisted_utc").str.starts_with("2024")
).head(1)
print(example_inactive.select(["ticker", "name", "primary_exchange", "delisted_utc"]))

# ============================================================================
# RESUMEN FINAL
# ============================================================================

print(f"\n{'='*80}")
print("RESUMEN FINAL - POBLACIÓN TARGET SMALL CAPS")
print(f"{'='*80}\n")

print(f"Total tickers: {len(df):,}")
print(f"   • Small Caps activos (< $2B): {len(df_active):,}")
print(f"   • Inactivos preservados: {len(df_inactive):,}")
print(f"   • Ratio activos/inactivos: {len(df_active)/len(df_inactive):.2f}")
print(f"\nAnti-survivorship bias: ✅ APLICADO")
print(f"Market cap range: ${mcap_stats['market_cap'][3]:,.0f} - ${mcap_stats['market_cap'][7]:,.0f}")
print(f"\nStatus: ✅ LISTO PARA FASE B (Ingesta Daily/Minute)")

VERIFICACIÓN: SMALL CAPS TARGET POPULATION (< $2B)

📊 RESUMEN GENERAL
   Total tickers: 6,405
   Total columnas: 25

   Small Caps activos: 3,105 (48.5%)
   Inactivos preservados: 3,300 (51.5%)

MARKET CAP - SMALL CAPS ACTIVOS (< $2B)

Activos con market_cap: 3,105 / 3,105

Estadísticas:
   Median: $     51,038,224
   Mean:   $              0
   Min:    $    512,354,312
   Max:    $    673,058,430

Distribución por rangos:
   Micro caps (< $300M)           1,747 ( 56.3%)
   Small caps ($300M-$1B)           863 ( 27.8%)
   Mid-small caps ($1B-$2B)         495 ( 15.9%)

DELISTED UTC - INACTIVOS PRESERVADOS (ANTI-SURVIVORSHIP BIAS)

Inactivos con delisted_utc: 3,300 / 3,300 (100.0%)

Distribución por año de delisting:
   2019: 351
   2020: 327
   2021: 467
   2022: 506
   2023: 696
   2024: 537
   2025: 416

DISTRIBUCIÓN POR EXCHANGE

XNAS:
   Total: 4,814 (75.2%)
   Activos: 2,464 | Inactivos: 2,350

XNYS:
   Total: 1,591 (24.8%)
   Activos: 641 | Inactivos: 950

COLUMNAS DISPONIBLES EN 

In [11]:
# ============================================================================
# Verificación: Small Caps Target Population - Análisis Detallado
# ============================================================================
import polars as pl

print("="*100)
print("VERIFICACIÓN: SMALL CAPS TARGET POPULATION (< $2B) - ANÁLISIS DETALLADO")
print("="*100)

# Cargar universo Small Caps
df = pl.read_parquet("processed/universe/smallcaps_universe_2025-11-01.parquet")

print(f"\n📊 RESUMEN GENERAL")
print(f"   Total tickers: {len(df):,}")
print(f"   Total columnas: {len(df.columns)}")

# Separar activos e inactivos
df_active = df.filter(pl.col("active") == True)
df_inactive = df.filter(pl.col("active") == False)

print(f"\n   Small Caps activos: {len(df_active):,} ({len(df_active)/len(df)*100:.1f}%)")
print(f"   Inactivos preservados: {len(df_inactive):,} ({len(df_inactive)/len(df)*100:.1f}%)")

# ============================================================================
# HEAD(2) - TODAS LAS COLUMNAS
# ============================================================================

print(f"\n{'='*100}")
print("📊 HEAD(2) - TODAS LAS COLUMNAS (Small Cap ACTIVO + Small Cap ACTIVO)")
print(f"{'='*100}\n")

# Obtener 2 ejemplos de Small Caps activos
df_examples = df_active.filter(~pl.col("market_cap").is_null()).head(2)

# Transponer para ver columnas como filas
print(f"shape: ({len(df.columns)}, 3)")
print(df_examples.transpose(include_header=True, header_name="column"))

# ============================================================================
# HEAD(2) - SOLO COLUMNAS CLAVE
# ============================================================================

print(f"\n{'='*100}")
print("📊 HEAD(2) - Solo columnas clave:")
print(f"{'='*100}\n")

key_cols = ["ticker", "name", "market_cap", "primary_exchange", "active", "type", "delisted_utc"]
key_cols_available = [c for c in key_cols if c in df.columns]

df_key = df_examples.select(key_cols_available)
print(f"shape: ({len(key_cols_available)}, 3)")
print(df_key.transpose(include_header=True, header_name="column"))

# ============================================================================
# COMPLETITUD DE CAMPOS
# ============================================================================

print(f"\n{'='*100}")
print("📊 COMPLETITUD DE CAMPOS:")
print(f"{'='*100}\n")

for col in df.columns:
    non_null = len(df.filter(~pl.col(col).is_null()))
    pct = non_null / len(df) * 100
    print(f"{col:<35}: {non_null:>6,} / {len(df):>6,} ({pct:>5.1f}%)")

# ============================================================================
# EJEMPLOS: ACTIVO vs INACTIVO
# ============================================================================

print(f"\n{'='*100}")
print("📊 COMPARACIÓN: SMALL CAP ACTIVO vs INACTIVO")
print(f"{'='*100}\n")

# Small Cap activo
print("EJEMPLO 1: Small Cap ACTIVO (< $2B)")
example_active = df_active.filter(~pl.col("market_cap").is_null()).head(1)
print(example_active.transpose(include_header=True, header_name="column"))

# Inactivo
print(f"\n{'='*100}")
print("EJEMPLO 2: INACTIVO PRESERVADO (delisted)")
df_inactive_example = df_inactive.filter(~pl.col("delisted_utc").is_null()).head(1)
print(df_inactive_example.transpose(include_header=True, header_name="column"))

# ============================================================================
# DISTRIBUCIÓN POR MARKET CAP
# ============================================================================

print(f"\n{'='*100}")
print("DISTRIBUCIÓN POR MARKET CAP - SMALL CAPS ACTIVOS")
print(f"{'='*100}\n")

df_active_with_mcap = df_active.filter(~pl.col("market_cap").is_null())

mcap_stats = df_active_with_mcap.select("market_cap").describe()
print(f"Estadísticas:")
print(f"   Median: ${mcap_stats['market_cap'][5]:>15,.0f}")
print(f"   Mean:   ${mcap_stats['market_cap'][1]:>15,.0f}")
print(f"   Min:    ${mcap_stats['market_cap'][3]:>15,.0f}")
print(f"   Max:    ${mcap_stats['market_cap'][7]:>15,.0f}")

print(f"\nDistribución por rangos:")
ranges = [
    ("Micro caps (< $300M)", 0, 300_000_000),
    ("Small caps ($300M-$1B)", 300_000_000, 1_000_000_000),
    ("Mid-small caps ($1B-$2B)", 1_000_000_000, 2_000_000_000),
]

for label, min_cap, max_cap in ranges:
    count = len(df_active_with_mcap.filter(
        (pl.col("market_cap") >= min_cap) & (pl.col("market_cap") < max_cap)
    ))
    pct = count / len(df_active_with_mcap) * 100
    print(f"   {label:<30} {count:>5,} ({pct:>5.1f}%)")

# ============================================================================
# DISTRIBUCIÓN POR AÑO DE DELISTING
# ============================================================================

print(f"\n{'='*100}")
print("DISTRIBUCIÓN POR AÑO DE DELISTING - INACTIVOS")
print(f"{'='*100}\n")

df_inactive_with_delisted = df_inactive.filter(~pl.col("delisted_utc").is_null())

print(f"Inactivos con delisted_utc: {len(df_inactive_with_delisted):,} / {len(df_inactive):,} ({len(df_inactive_with_delisted)/len(df_inactive)*100:.1f}%)")

if len(df_inactive_with_delisted) > 0:
    df_delisted_years = (
        df_inactive_with_delisted
        .with_columns(
            pl.col("delisted_utc").str.slice(0, 4).alias("delisted_year")
        )
        .group_by("delisted_year")
        .len()
        .sort("delisted_year")
    )
    
    print(f"\nDistribución por año de delisting:")
    for row in df_delisted_years.iter_rows(named=True):
        print(f"   {row['delisted_year']}: {row['len']:,}")

# ============================================================================
# RESUMEN FINAL
# ============================================================================

print(f"\n{'='*100}")
print("✅ RESUMEN FINAL - POBLACIÓN TARGET SMALL CAPS")
print(f"{'='*100}\n")

print(f"Total tickers: {len(df):,}")
print(f"   • Small Caps activos (< $2B): {len(df_active):,} ({len(df_active)/len(df)*100:.1f}%)")
print(f"   • Inactivos preservados: {len(df_inactive):,} ({len(df_inactive)/len(df)*100:.1f}%)")
print(f"   • Ratio activos/inactivos: {len(df_active)/len(df_inactive):.2f}")
print(f"\nAnti-survivorship bias: ✅ APLICADO")
print(f"Market cap range: ${mcap_stats['market_cap'][3]:,.0f} - ${mcap_stats['market_cap'][7]:,.0f}")
print(f"\nDistribución por exchange:")
print(f"   • NASDAQ (XNAS): {len(df.filter(pl.col('primary_exchange') == 'XNAS')):,} (75.2%)")
print(f"   • NYSE (XNYS): {len(df.filter(pl.col('primary_exchange') == 'XNYS')):,} (24.8%)")
print(f"\nStatus: ✅ LISTO PARA FASE B (Ingesta Daily/Minute)")

VERIFICACIÓN: SMALL CAPS TARGET POPULATION (< $2B) - ANÁLISIS DETALLADO

📊 RESUMEN GENERAL
   Total tickers: 6,405
   Total columnas: 25

   Small Caps activos: 3,105 (48.5%)
   Inactivos preservados: 3,300 (51.5%)

📊 HEAD(2) - TODAS LAS COLUMNAS (Small Cap ACTIVO + Small Cap ACTIVO)

shape: (25, 3)
shape: (25, 3)
┌────────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┐
│ column                         ┆ column_0                        ┆ column_1                        │
╞════════════════════════════════╪═════════════════════════════════╪═════════════════════════════════╡
│ ticker                         ┆ AACB                            ┆ AAM                             │
│ name                           ┆ Artius II Acquisition Inc. Cla… ┆ AA Mission Acquisition Corp.    │
│ market                         ┆ stocks                          ┆ stocks                          │
│ locale                         ┆ us                             

## Enriquecimiento de Splits & Dividends (Datos globales)

**Objetivo**: Obtener eventos corporativos históricos (splits, dividends) para ajustes de precio.  
**Script**: [scripts/ingest_splits_dividends.py](../scripts/00_universe_ingest/ingest_splits_dividends.py)    
**Fuente de datos**: Polygon `/v3/reference/splits` y `/v3/reference/dividends` (sin filtros) 
**IMPORTANTE**: Este es un backup GLOBAL de todos los eventos corporativos disponibles en Polygon. Los datos se filtrarán posteriormente para nuestro universo de 6,405 tickers Small Caps.

In [13]:
# =============================================================================
# VERIFICACIÓN: SPLITS & DIVIDENDS FILTRADOS (Solo 6,405 tickers Small Caps)
# =============================================================================

import polars as pl
from pathlib import Path

print("="*100)
print("VERIFICACIÓN SPLITS & DIVIDENDS - UNIVERSO SMALL CAPS (6,405 tickers)")
print("="*100)

# -----------------------------------------------------------------------------
# RESUMEN EJECUTIVO
# -----------------------------------------------------------------------------
summary = pl.read_csv("processed/corporate_actions/summary.csv")
print("\n📊 RESUMEN EJECUTIVO:")
print("-" * 100)
print(summary)

# -----------------------------------------------------------------------------
# SPLITS FILTRADOS
# -----------------------------------------------------------------------------
print("\n\n📊 SPLITS FILTRADOS (processed/corporate_actions/splits/year=*/)")
print("-" * 100)

splits_files = sorted(Path("processed/corporate_actions/splits").glob("year=*/splits.parquet"))
df_splits = pl.concat([pl.read_parquet(f) for f in splits_files])

print(f"Total splits:              {len(df_splits):,}")
print(f"Total columns:             {len(df_splits.columns)}")
print(f"Tickers únicos:            {df_splits['ticker'].n_unique():,}")
print(f"Período:                   {df_splits['execution_date'].min()} - {df_splits['execution_date'].max()}")

# Top tickers con más splits
print("\n📌 Top 10 tickers con más splits:")
top_splits = df_splits.group_by("ticker").len().sort("len", descending=True).head(10)
print(top_splits)

# Distribución por año (Top 10)
print("\n📅 Top 10 años con más splits:")
df_splits_year = df_splits.with_columns(pl.col("execution_date").str.slice(0, 4).alias("year"))
year_dist = df_splits_year.group_by("year").len().sort("len", descending=True).head(10)
print(year_dist)

# Head transpuesto
print("\n📋 HEAD(3) - Ejemplo de splits:")
print(df_splits.select(["ticker", "execution_date", "split_from", "split_to", "ratio"]).head(3))

# -----------------------------------------------------------------------------
# DIVIDENDS FILTRADOS
# -----------------------------------------------------------------------------
print("\n\n💰 DIVIDENDS FILTRADOS (processed/corporate_actions/dividends/year=*/)")
print("-" * 100)

dividends_files = sorted(Path("processed/corporate_actions/dividends").glob("year=*/dividends.parquet"))
df_dividends = pl.concat([pl.read_parquet(f) for f in dividends_files])

print(f"Total dividends:           {len(df_dividends):,}")
print(f"Total columns:             {len(df_dividends.columns)}")
print(f"Tickers únicos:            {df_dividends['ticker'].n_unique():,}")
print(f"Período:                   {df_dividends['ex_dividend_date'].min()} - {df_dividends['ex_dividend_date'].max()}")

# Top tickers con más dividends
print("\n📌 Top 10 tickers con más dividends:")
top_divs = df_dividends.group_by("ticker").len().sort("len", descending=True).head(10)
print(top_divs)

# Distribución por tipo
print("\n📊 Distribución por tipo:")
type_dist = df_dividends.group_by("dividend_type").len().sort("len", descending=True)
print(type_dist)

# Estadísticas de cash_amount
print("\n💵 Estadísticas de cash_amount:")
cash_stats = df_dividends.select("cash_amount").describe()
print(cash_stats)

# Distribución por año (Top 10)
print("\n📅 Top 10 años con más dividends:")
df_divs_year = df_dividends.with_columns(pl.col("ex_dividend_date").str.slice(0, 4).alias("year"))
year_dist_div = df_divs_year.group_by("year").len().sort("len", descending=True).head(10)
print(year_dist_div)

# Head transpuesto
print("\n📋 HEAD(3) - Ejemplo de dividends:")
print(df_dividends.select(["ticker", "ex_dividend_date", "cash_amount", "dividend_type", "frequency"]).head(3))

# -----------------------------------------------------------------------------
# COBERTURA DEL UNIVERSO
# -----------------------------------------------------------------------------
print("\n\n🎯 COBERTURA DEL UNIVERSO (6,405 tickers)")
print("-" * 100)

df_universe = pl.read_parquet("processed/universe/smallcaps_universe_2025-11-01.parquet")
total_tickers = len(df_universe)
activos = len(df_universe.filter(pl.col("active") == True))
inactivos = len(df_universe.filter(pl.col("active") == False))

tickers_con_splits = df_splits["ticker"].n_unique()
tickers_con_dividends = df_dividends["ticker"].n_unique()

print(f"Universo total:            {total_tickers:,} tickers")
print(f"  • Activos:               {activos:,} ({activos/total_tickers*100:.1f}%)")
print(f"  • Inactivos:             {inactivos:,} ({inactivos/total_tickers*100:.1f}%)")
print(f"\nTickers con splits:        {tickers_con_splits:,} ({tickers_con_splits/total_tickers*100:.1f}%)")
print(f"Tickers con dividends:     {tickers_con_dividends:,} ({tickers_con_dividends/total_tickers*100:.1f}%)")

print("\n" + "="*100)
print("✅ VERIFICACIÓN COMPLETADA")
print("="*100)

VERIFICACIÓN SPLITS & DIVIDENDS - UNIVERSO SMALL CAPS (6,405 tickers)

📊 RESUMEN EJECUTIVO:
----------------------------------------------------------------------------------------------------
shape: (11, 2)
┌───────────────────────────┬─────────┐
│ metric                    ┆ value   │
╞═══════════════════════════╪═════════╡
│ universe_size             ┆ 6405    │
│ splits_global             ┆ 26696   │
│ splits_filtrado           ┆ 3420    │
│ splits_pct_retenido       ┆ 12.81%  │
│ tickers_con_splits        ┆ 2009    │
│ tickers_con_splits_pct    ┆ 31.4%   │
│ dividends_global          ┆ 1881581 │
│ dividends_filtrado        ┆ 71291   │
│ dividends_pct_retenido    ┆ 3.79%   │
│ tickers_con_dividends     ┆ 1768    │
│ tickers_con_dividends_pct ┆ 27.6%   │
└───────────────────────────┴─────────┘


📊 SPLITS FILTRADOS (processed/corporate_actions/splits/year=*/)
----------------------------------------------------------------------------------------------------
Total splits:            